# Data Science en Entreprise

## API pôle emploi

Autorisation d’accès à l’api des offres d’emplois

Compte pole emploi :    
username : mohammed.didi@outlook.com    
password : Projet_Data1



In [12]:
import requests


def obtenir_token(client_id, client_secret):
    """
    Cette fonction obtient un token d'accès pour l'API de Pôle Emploi.
    """
    url_token = 'https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=/partenaire'
    auth_data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
        'scope': 'api_offresdemploiv2 o2dsoffre'
    }
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}

    response = requests.post(url_token, data=auth_data, headers=headers)

    if response.status_code != 200:
        raise Exception(f"Erreur lors de la requête : {response.text}")
    else:
        data = response.json()
        return data['access_token']
    
# Utilisation de la fonction
client_id = 'PAR_projetdata_29c785675807663802eabdf57858fc4e2ecd4eba4de8d05eee293bfdcacefae5'
client_secret = '8a48c4726bb2d33cafb7a636703a3b0ad70e4098b525063d2a99387a95a76b70'
token = obtenir_token(client_id, client_secret)

In [29]:
import requests
from datetime import datetime
import pandas as pd
import json

def rechercher_offres(token, mot_cle, mois, annee, nb_offres=10):
    """
    Recherche les offres d'emploi dans le domaine des RH pour un mois et une année spécifiques.
    """
    url_offres = 'https://api.emploi-store.fr/partenaire/offresdemploi/v2/offres/search'
    headers = {'Authorization': f'Bearer {token}'}

    # Calculer les dates de début et de fin du mois de novembre
    debut_mois = f"{annee}-{mois:02d}-01T00:00:00Z"
    fin_mois = f"{annee}-{mois:02d}-30T00:00:00Z"  # Novembre a 30 jours

    params = {
        'motsCles': mot_cle,  # Mots-clés pour le domaine des Ressources Humaines
        'range': f'0-{nb_offres-1}',  # Les 10 premières offres
        'minCreationDate': debut_mois,
        'maxCreationDate': fin_mois
    }

    response = requests.get(url_offres, headers=headers, params=params)

    try:
        return response.json()
    except:
        raise Exception(f"Erreur lors de la requête : {response.text}")



# Recherche des offres en RH pour le mois de novembre
offres_rh_novembre = rechercher_offres(token, 'RH',11, 2023)  # Exemple pour novembre 2023

# jsoon to dataframe
df = pd.DataFrame.from_dict(offres_rh_novembre['resultats'])


In [30]:
df

,id,intitule,description,dateCreation,dateActualisation,lieuTravail,romeCode,romeLibelle,appellationlibelle,entreprise,...,qualificationCode,qualificationLibelle,codeNAF,secteurActivite,secteurActiviteLibelle,origineOffre,offresManqueCandidats,deplacementCode,deplacementLibelle,qualitesProfessionnelles
0,165PTDQ,ASSISTANTE RESSOURCES HUMAINES(H/F) (H/F),"Notre agence recrute, un.e Assistant.e Ressour...",2023-11-29T18:15:18.000Z,2023-12-01T02:41:25.000Z,"{'libelle': '34 - MONTPELLIER', 'latitude': 43...",M1501,Assistanat en ressources humaines,Assistant / Assistante de gestion du personnel,"{'nom': 'CRIT INTERIM', 'entrepriseAdaptee': F...",...,6,Employé qualifié,78.20Z,78,Activités des agences de travail temporaire,"{'origine': '1', 'urlOrigine': 'https://candid...",False,NaN,NaN,NaN
1,165PSWD,25819 - expert rh (H/F),Le poste :\nVotre agence PROMAN de BUSSY-SAINT...,2023-11-29T18:00:58.000Z,2023-12-01T02:41:25.000Z,"{'libelle': '77 - MONTEVRAIN', 'latitude': 48....",M1503,Management des ressources humaines,Coordinateur / Coordinatrice ressources humaines,"{'nom': 'PROMAN', 'url': 'https://www.proman-e...",...,5,Employé non qualifié,78.20Z,78,Activités des agences de travail temporaire,"{'origine': '1', 'urlOrigine': 'https://candid...",False,NaN,NaN,NaN
2,165PSPM,Assistant Ressources Humaines (h/f),"Au sein d?Adecco, dans le cadre d?un remplacem...",2023-11-29T17:53:28.000Z,2023-12-01T02:41:25.000Z,"{'libelle': '50 - ST LO', 'latitude': 49.11431...",M1501,Assistanat en ressources humaines,Assistant / Assistante ressources humaines,"{'nom': 'ADECCO FRANCE', 'description': 'Premi...",...,6,Employé qualifié,78.20Z,78,Activités des agences de travail temporaire,"{'origine': '1', 'urlOrigine': 'https://candid...",False,NaN,NaN,NaN
3,165PRXH,Gestionnaire administratif / administrative re...,Si on vous disait qu'aujourd'hui il est possib...,2023-11-29T17:34:13.000Z,2023-12-01T02:41:25.000Z,"{'libelle': '69 - LYON 06', 'latitude': 45.772...",M1501,Assistanat en ressources humaines,Gestionnaire administratif(ve) ressources huma...,"{'nom': 'LEYTON FRANCE', 'entrepriseAdaptee': ...",...,6,Employé qualifié,70.22Z,70,Conseil pour les affaires et autres conseils d...,"{'origine': '1', 'urlOrigine': 'https://candid...",False,1,Jamais,"[{'libelle': 'Travailler en équipe', 'descript..."
4,165PRNR,Assistant paie et RH (H/F),L'Association ASPEC est gestionnaire d'établis...,2023-11-29T17:22:38.000Z,2023-11-29T17:22:39.000Z,"{'libelle': '61 - MORTAGNE AU PERCHE', 'latitu...",M1203,Comptabilité,Assistant / Assistante paie,"{'nom': 'ASSOCIATION ASPEC', 'description': 'P...",...,6,Employé qualifié,88.10C,88,Aide par le travail,"{'origine': '1', 'urlOrigine': 'https://candid...",False,1,Jamais,"[{'libelle': 'Travailler en équipe', 'descript..."
5,165PQSQ,Assistant(e) RH - Formation (H/F),"Intégrer TSO, c'est rejoindre une filiale du G...",2023-11-29T17:01:01.000Z,2023-12-01T02:41:25.000Z,"{'libelle': '77 - CHELLES', 'latitude': 48.886...",M1501,Assistanat en ressources humaines,Assistant / Assistante de service formation,"{'nom': 'TSO', 'entrepriseAdaptee': False}",...,6,Employé qualifié,42.12Z,42,Construction de voies ferrées de surface et so...,"{'origine': '1', 'urlOrigine': 'https://candid...",False,1,Jamais,"[{'libelle': 'Travailler en équipe', 'descript..."
6,165PMSN,Chargé de recrutement/RH sur site industriel (...,"Être responsable de compte chez nous, c'est vi...",2023-11-29T16:05:26.000Z,2023-12-01T02:41:25.000Z,"{'libelle': '61 - CALIGNY', 'latitude': 48.804...",M1502,Développement des ressources humaines,Chargé / Chargée de recrutement,"{'nom': 'GROUPE RANDSTAD FRANCE', 'description...",...,8,Agent de maîtrise,70.10Z,70,Activités des sièges sociaux,"{'origine': '1', 'urlOrigine': 'https://candid...",False,NaN,NaN,"[{'libelle': 'Faire preuve de réactivité', 'de..."
7,165PMMY,Assistant ressources humaines (F/H),Nous recherchons pour le compte de notre clien...,2023-11-29T16:01:58.000Z,2023-11-30T14:30:13.000Z,"{'libelle': '13 - VITROLLES'

In [31]:
df.columns

Index(['id', 'intitule', 'description', 'dateCreation', 'dateActualisation',
       'lieuTravail', 'romeCode', 'romeLibelle', 'appellationlibelle',
       'entreprise', 'typeContrat', 'typeContratLibelle', 'natureContrat',
       'experienceExige', 'experienceLibelle', 'formations', 'permis',
       'competences', 'salaire', 'dureeTravailLibelle',
       'dureeTravailLibelleConverti', 'alternance', 'contact', 'nombrePostes',
       'accessibleTH', 'qualificationCode', 'qualificationLibelle', 'codeNAF',
       'secteurActivite', 'secteurActiviteLibelle', 'origineOffre',
       'offresManqueCandidats', 'deplacementCode', 'deplacementLibelle',
       'qualitesProfessionnelles'],
      dtype='object')

In [33]:
# save df to csv on ../data

df.to_csv('../data/offres_rh_novembre.csv', index=False)
